# Pandas에서 누락된 데이터

In [1]:
#None: 파이썬의 누락된 데이터
import numpy as np
import pandas as pd

In [2]:
#None은 파이썬 객체이므로 데이터 타입이 'object'인 배열에서만 사용할 수 있다
vals1=np.array([1,None,3,4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object','int']:
    print("dtype =",dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()
#None 값을 가진 배열에서 sum()이나 mmin() 같은 집계연산을 하면 일반적으로 오류가 발생
#예를 들어 vals1.sum()실행 할 경우 오류 발생

dtype = object
117 ms ± 4.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
4.09 ms ± 24.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [4]:
#NaN: 누락된 숫자 데이터
#특수 부동 소수점 값을 가진다
vals2=np.array([1,np.nan,3,4])
vals2.dtype

dtype('float64')

In [5]:
#어떤 연산이든 상관없이 NaN이 포함된 산술 연산의 결과는 또 다른 NaN이 된다
1+np.nan

nan

In [6]:
#값을 집계하는 방법은 정의 되어있지만(오류는 없다) 그것이 항상 유용하지는 않다
vals2.sum() , vals2.min() , vals2.max()

(nan, nan, nan)

In [7]:
#누락된 값을 무시하는 집계 연산 존재
np.nansum(vals2) , np.nanmin(vals2) , np.nanmax(vals2)

(8.0, 1.0, 4.0)

In [8]:
#Pandas에서 NaN과 None
#정수 배열의 값을 np.nan으로 설정하면 NA를 수용할 수 있도록 부동 소수점 타입으로 자동 상향 변환한다
pd.Series([1,np.nan,2,None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [9]:
x=pd.Series(range(2),dtype=int)
x

0    0
1    1
dtype: int32

In [10]:
#자동으로 None을 NaN값으로 변환한다
x[0]=None
x

0    NaN
1    1.0
dtype: float64

# 널 값 연산하기

In [11]:
#널 값 탐지
data=pd.Series([1,np.nan,'hello',None])
data.isnull()      #누락값을 가리키는 부울 마스크를 생성

0    False
1     True
2    False
3     True
dtype: bool

In [12]:
#부울 마스크는 Series 나 DataFrame 인덱스로 직접 사용 가능
data[data.notnull()]    #isnull()의 역

0        1
2    hello
dtype: object

In [13]:
#널 값 제거하기
data.dropna()    #데이터에 필터를 적용한 버전을 반환, NA값 제거하기

0        1
2    hello
dtype: object

In [14]:
df=pd.DataFrame([[1, np.nan, 2],
                [2, 3, 5],
                [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [15]:
#DataFrame의 경우 단일 값만 삭제할 수 없으며, 전체 행이나 전체 열을 삭제하는 것만 가능
#기본적으로 dropna()는 널 값이 있는 모든 행을 삭제
df.dropna()

,0,1,2
1,2.0,3.0,5


In [16]:
#널 값을 포함하는 모든 열을 삭제할 수 있다
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [17]:
#how='any'로 널 값을 포함하는 행이나 열을 모두 삭제
#how='all'로 모두 널 값인 행이나 열만 삭제
df[3]=np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [18]:
df.dropna(axis='columns',how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [19]:
#thresh 매개변수로 행이나 열에서 널이 아닌 값이 최소 몇 개가 있어야 하는지 지정 가능
df.dropna(axis='rows',thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


In [20]:
#널 값 채우기
data=pd.Series([1,np.nan,2,None,3],index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [21]:
data.fillna(0)   #누락 값을 채우거나 전가된 데이터 사본을 반환

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [22]:
#forward-fill : 이전 값으로 채우기
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [23]:
#back-fill : 다음에 오는 값으로 채우기
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [24]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [25]:
df.fillna(method='ffill',axis=1)    #뒤의 값을 채울 때 이전 값을 사용할 수 없다면 NA 값은 그대로 남는다

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
